In [1]:
import sys
import json

import time
sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/dgl_graphsage')

from utils import load_data
from train_updated import train

import torch
import torch.nn.functional as F
from collections import defaultdict
import pandas as pd
import os

Using backend: pytorch


In [2]:
import dgl

In [3]:
%%time
with open('config/data-params.json') as fh:
    data_cfg = json.load(fh)
feat_data, adj_list, dgl_G, uri_map = load_data(**data_cfg)

Loading graph data...
Loading feature data...
CPU times: user 1min 11s, sys: 13.7 s, total: 1min 25s
Wall time: 1min 24s


In [4]:
%%time
with open('config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred, train_ids, val_G, val_neg_G, embed= train(dgl_G, feat_data, adj_list, **model_cfg)

Train pos edge: 13206790
Validation pos edge: 259232
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.6960936784744263
In epoch 1 batch 6, loss: 0.6225101947784424
In epoch 1 batch 11, loss: 0.5545179843902588
In epoch 1 batch 16, loss: 0.44346365332603455
In epoch 1 batch 21, loss: 0.27571821212768555

Epoch 1 AUC:  0.5113454638963366
In epoch 2 batch 1, loss: 0.1918533742427826
In epoch 2 batch 6, loss: 0.18685542047023773
In epoch 2 batch 11, loss: 0.21220900118350983
In epoch 2 batch 16, loss: 0.17590132355690002
In epoch 2 batch 21, loss: 0.17722798883914948

Epoch 2 AUC:  0.535905347456916
In epoch 3 batch 1, loss: 0.1836790293455124
In epoch 3 batch 6, loss: 0.17877602577209473
In epoch 3 batch 11, loss: 0.17430400848388672
In epoch 3 batch 16, loss: 0.18251779675483704
In epoch 3 batch 21, loss: 0.16607895493507385

Epoch 3 AUC:  0.5616526134095625
In epoch 4 batch 1, loss: 0.17148125171661377
In epoch 4 batch 6, loss: 0.1789313703775406
In epoch 4 batch 11, los

In [35]:
model = model.to('cpu')
val_G = val_G.to('cpu')
pred = pred.to('cpu')
embed = embed.to('cpu')
val_neg_G = val_neg_G.to('cpu')

In [38]:
z = model(dgl_G, feat_data)

In [39]:
preds = pred(dgl_G, z)

In [40]:
preds

tensor([2.8106, 2.8177, 2.8171,  ..., 3.1957, 3.1985, 3.1889],
       grad_fn=<SqueezeBackward1>)

In [42]:
dgl_G.edges()[:10]

(tensor([ 10959,  10959,  10959,  ...,   4036,   4036, 163739]),
 tensor([137484,  11768,  25799,  ..., 163739, 150945, 150945]))

In [103]:
def wowa(thelist):
    eligible = []
    for x in thelist:
        nums = pd.Series(x.strip('mpd.slice.json').split('-')).astype(int)
        if nums[0] <= 9999:
            eligible.append(x)
    return eligible

In [104]:
data_path = (os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/'))

In [218]:
file_samp = np.random.choice(wowa(pd.Series(os.listdir(os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/')))), replace=True)
fname = os.path.join(data_path, file_samp)
with open(fname) as f:
    data = json.load(f)
    item = np.random.choice(data['playlists'])
item

{'name': 'Old Rap',
 'collaborative': 'false',
 'pid': 4692,
 'modified_at': 1373500800,
 'num_tracks': 7,
 'num_albums': 7,
 'num_followers': 1,
 'tracks': [{'pos': 0,
   'artist_name': 'Lupe Fiasco',
   'track_uri': 'spotify:track:4NTWZqvfQTlOMitlVn6tew',
   'artist_uri': 'spotify:artist:01QTIT5P1pFP3QnnFSdsJf',
   'track_name': 'The Show Goes On',
   'album_uri': 'spotify:album:1j0apvEvaWbTmlZpKsfr2D',
   'duration_ms': 239613,
   'album_name': 'Lasers'},
  {'pos': 1,
   'artist_name': 'Wiz Khalifa',
   'track_uri': 'spotify:track:3GzjG6Qah8cplpj5YAURLK',
   'artist_uri': 'spotify:artist:137W8MRPWKqSmrBGDBFSop',
   'track_name': 'Say Yeah',
   'album_uri': 'spotify:album:2pqfPXKjLSTMGsCq3HiQ2p',
   'duration_ms': 241613,
   'album_name': 'Say Yeah'},
  {'pos': 2,
   'artist_name': 'Big Sean',
   'track_uri': 'spotify:track:3ZAMtgYJFoHwJjFkhkXqKr',
   'artist_uri': 'spotify:artist:0c173mlxpT3dSFRgMO8XPh',
   'track_name': 'Dance (A$$) Remix',
   'album_uri': 'spotify:album:2fRLnox9PQ

In [219]:
len(item['tracks'])#['track_name']

7

In [220]:
original_tracks = []
for i in range(len(item['tracks'])):
    name = item['tracks'][i]['track_name']+'---'+item['tracks'][i]['artist_name']
    original_tracks.append(name)

In [221]:
original_tracks

['The Show Goes On---Lupe Fiasco',
 'Say Yeah---Wiz Khalifa',
 'Dance (A$$) Remix---Big Sean',
 'No Hands (feat. Roscoe Dash and Wale) - Explicit Album Version---Waka Flocka Flame',
 "I'm On One---DJ Khaled",
 'Break Ya Neck---Busta Rhymes',
 'Lose My Mind---Jeezy']

In [222]:
seeds = []
for i in item['tracks']:
    uri = i['track_uri'].split(':')[-1]
    seeds.append(uri)

In [223]:
seeds

['4NTWZqvfQTlOMitlVn6tew',
 '3GzjG6Qah8cplpj5YAURLK',
 '3ZAMtgYJFoHwJjFkhkXqKr',
 '03tqyYWC9Um2ZqU0ZN849H',
 '608a1wIsSd5KzMEqm1O7w3',
 '0svl7cK07gK1ia5ainczU5',
 '0Ft6hwGWdCdujdK6TOfBV7']

In [224]:
score_dict = defaultdict(dict)
for s in seeds:
    s = uri_map[s]
    _, candidates = dgl_G.out_edges(s, form='uv')
    s_embed = z[s].unsqueeze(dim=0)
    edge_embeds = [torch.cat([s_embed, z[c.item()].unsqueeze(dim=0)],1) for c in candidates]
    edge_embeds = torch.cat(edge_embeds, 0)
    scores = pred.W2(F.relu(pred.W1(edge_embeds))).squeeze(1)
    val = list(zip(candidates.detach().numpy(), scores.detach().numpy()))
    val.sort(key=lambda x:x[1], reverse=True)
    
    score_dict[s] = val[0]
#     to_keep = 0
#     inc = 0
#     #in_playlist = True
#     while True:
#         if listed[val[inc]] not in seeds:
#             score_dict[s] = val[inc]
#             break
#         else:
#             inc += 1
    

In [225]:
score_dict#[144044].sort(key=lambda x:x[1], reverse=True)

defaultdict(dict,
            {95807: (90872, 3.2378929),
             71647: (114891, 3.2297347),
             77993: (140898, 3.2568882),
             1376: (40663, 3.234962),
             131076: (14564, 3.252432),
             19467: (151598, 3.240677),
             5821: (14564, 3.2365043)})

In [226]:
listed = list(uri_map)
uri_recs = []
for i in score_dict.keys():
    cur_uri = listed[score_dict[i][0]]
    uri_recs.append(cur_uri)

In [227]:
uri_recs

['49hqWRCETPpqIwQ3zzL74G',
 '5Fv85gWgP0Rgkp8JnNjvLp',
 '6S1bKWlMqa0w89RxnV7ne8',
 '1r9hm3LbOD9iUaweV633zo',
 '0esX69yFN8t8tbN9e0VWSa',
 '6wUhjJzN7cvzoOX3Zpl8B0',
 '0esX69yFN8t8tbN9e0VWSa']

In [121]:
from src.api.spotifyAPI import SpotifyAPI

In [126]:
client_id = 'ad2536ed7a914d66b89b80fb3a500787'
client_secret = '8c5f45fb008d4bc5bf909ec46d076b65'

spotify = SpotifyAPI(client_id, client_secret)

In [140]:
def get_data(query, api, num):
    chunk = api.get_resource(query, 'tracks', 'v1')
    
    #json_object = json.dumps(chunk, indent = 4)
    
    return chunk

In [228]:
%%time
rec_track_names = []
for i in uri_recs:
    one = get_data(i, spotify, 1)
    trackname = one['tracks'][0]['name']
    
    firstartist = one['tracks'][0]['artists'][0]['name']
    
    the_rec = trackname+'---'+firstartist
    rec_track_names.append(the_rec)
    
    
    time.sleep(5)

CPU times: user 259 ms, sys: 41.6 ms, total: 300 ms
Wall time: 36.4 s


In [229]:
original_tracks

['The Show Goes On---Lupe Fiasco',
 'Say Yeah---Wiz Khalifa',
 'Dance (A$$) Remix---Big Sean',
 'No Hands (feat. Roscoe Dash and Wale) - Explicit Album Version---Waka Flocka Flame',
 "I'm On One---DJ Khaled",
 'Break Ya Neck---Busta Rhymes',
 'Lose My Mind---Jeezy']

In [230]:
rec_track_names

['Bout Shit---Lud Foe',
 'Nights Like This---Nyzzy Nyce',
 'Phone Message (Interlude)---The Last Mr. Bigg',
 '50 Chicks---Migos',
 'Young Money---Tyga',
 "Don't Ever Fucking Question That---Atmosphere",
 'Young Money---Tyga']

In [ ]:
fname = (os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.0-999.json'))
'''
Get int ids for each track in playlist
'''
def pull_tracks(fname, label_to_int):
    with open(fname) as f:
        data = json.load(f)
        item = np.random.choice(data['playlists']) # Get a random playlist to recommend for
    
    # Get int values from track labels
    int_vals = []
    for track in item['tracks']:
        code = track['track_uri'].replace('spotify:track:', '')
        int_vals.append(label_to_int[code])
        
    return int_vals
'''
Create recommendations for each song in the playlist.
'''
def recommend(int_vals, model):
    recs = defaultdict(list) # dictionary of recommendations in form {'id': '(rec1, rank), '(rec2, rank)}
    for i in int_vals:
        preds = model.predict(i)
        for j in preds: # keep only predictions not already in playlist
            if j['id'] not in int_vals:
                recs[i].append((j['id'], j['rank']))
                
    return recs
'''
Rank all the recommendations, taking the highest recommended song
'''
def rank(recs):
    final = []
    for i in recs.keys():
        recs[i].sort(key = lambda x:x[1], reverse=True)
        # Get highest rated recommendation for each song
        final.append(recs[i][0])
        
    # Sort by best ranks
    final.sort(key = lambda x:x[1], reverse=True)
    ranked = list(zip(*final))[0]
    return ranked
'''
Decode to song labels
'''
def decode(ranked, int_to_label):
    decoded = []
    for i in ranked:
        decoded.append(int_to_label[i])
    return decoded